In [ ]:
# fasttext

In [ ]:
import pandas as pd
# read_csv方法中有3个参数，第1个参数是加载文本文件的路径，第2个关键字参数sep是分隔符，第3个关键字参数header是文本文件的第1行是否为字段名
train_df = pd.read_csv('data/text_train.txt', sep='\t', header=None)
train_df.columns = ['分类', '内容']
train_df.head() # 打印训练集前5行

In [ ]:
# 查看训练集每个分类的名字以及样本数量
for name, group in train_df.groupby(train_df.columns[0]):
    print(name,len(group))

In [ ]:
# 加载测试集并查看每个分类的名字以及样本数量
test_df = pd.read_csv('data/text_test.txt', sep='\t', header=None)
for name, group in test_df.groupby(test_df.columns[0]):
    print(name, len(group))

In [ ]:
# 文本分类不是任何词都是重要的判断语句，针对一些没有任何代表意义的词（的、了、么、吗）应该去掉，这种词容易对模型进行干扰
with open('data/stopwords.txt', encoding='utf8') as file:
    line_list = file.readlines()
    stopword_list = [k.strip() for k in line_list]
    stopword_set = set(stopword_list)
    print('停顿词列表，即变量stopword_list中共有%d个元素' %len(stopword_list))
    print('停顿词集合，即变量stopword_set中共有%d个元素' %len(stopword_set))

In [ ]:
def word_cut(content, stopword_set):
    cutWords = [k for k in jieba.cut(content, True) if k not in stopword_set]
    return " ".join(cutWords)

In [ ]:
fasttext_train_df = train_df.copy()
fasttext_train_df.shape

In [ ]:
fasttext_train_df.head()

In [ ]:
# 训练样本内容分词
fasttext_train_df["内容"] = fasttext_train_df["内容"].map(lambda x: word_cut(x, stopword_set))
fasttext_train_df.head()

In [ ]:
# label映射
label_map ={}
label_map_idx = {}
i = 0
for name, group in fasttext_train_df.groupby(test_df.columns[0]):
    i += 1
    print(name, len(group))
    label_map[name] = "__label__" + str(i)
    label_map_idx["__label__" + str(i)] = name
label_map

In [ ]:
label_map_idx

In [ ]:
# 标签映射
# label_map = test_df.groupby(test_df.columns[0])

In [ ]:
fasttext_train_df["分类"] = fasttext_train_df["分类"].map(lambda x: label_map[x])

In [ ]:
fasttext_train_df.head(5)

In [ ]:
# 测试集
fasttext_test_df = test_df.copy()
fasttext_test_df.shape

In [ ]:
# 测试集，内容分词
fasttext_test_df["内容"] = fasttext_test_df["内容"].map(lambda x: word_cut(x, stopword_set))
fasttext_test_df.head()

In [ ]:
# 测试集，标签映射
fasttext_test_df["分类"] = fasttext_test_df["分类"].map(lambda x: label_map[x])

In [ ]:
fasttext_test_df.head(5)

In [ ]:
# dataframe互换两列的位置
test_mid = fasttext_test_df['内容']
fasttext_test_df.drop(labels=['内容'], axis=1,inplace = True)
fasttext_test_df.insert(0, '内容', test_mid)
fasttext_test_df.head(5)

In [ ]:
# dataframe互换两列的位置
train_mid = fasttext_train_df['内容']
fasttext_train_df.drop(labels=['内容'], axis=1,inplace = True)
fasttext_train_df.insert(0, '内容', train_mid)
fasttext_train_df.head(5)

In [ ]:
fasttext_test_df.to_csv("data/fasttext_test_df.txt", sep="\t", index=False, header=None)

In [ ]:
fasttext_train_df.to_csv("data/fasttext_train_df.txt", sep="\t", index=False, header=None)

In [ ]:
import fasttext
"""
  训练一个监督模型, 返回一个模型对象

  @param input:           训练数据文件路径
  @param lr:              学习率
  @param dim:             向量维度
  @param ws:              cbow模型时使用
  @param epoch:           次数
  @param minCount:        词频阈值, 小于该值在初始化时会过滤掉
  @param minCountLabel:   类别阈值，类别小于该值初始化时会过滤掉
  @param minn:            构造subword时最小char个数
  @param maxn:            构造subword时最大char个数
  @param neg:             负采样
  @param wordNgrams:      n-gram个数
  @param loss:            损失函数类型, softmax, ns: 负采样, hs: 分层softmax
  @param bucket:          词扩充大小, [A, B]: A语料中包含的词向量, B不在语料中的词向量
  @param thread:          线程个数, 每个线程处理输入数据的一段, 0号线程负责loss输出
  @param lrUpdateRate:    学习率更新
  @param t:               负采样阈值
  @param label:           类别前缀
  @param pretrainedVectors: 预训练的词向量文件路径, 如果word出现在文件夹中初始化不再随机
  @return model object
"""
classifier = fasttext.train_supervised(input='data/fasttext_train_df.txt', dim=100, epoch=10,
                                         lr=0.1, wordNgrams=2, loss='softmax', label="__label__")

In [ ]:
result = classifier.test('data/fasttext_test_df.txt')
print(result)

In [ ]:
classifier.predict(["奥运 明星 明星写真 写真 写真集 集锦 曝光 　 展现 健康 时尚 图 来源 人民"])